# 명탐정 코난 매니아 판별기 (LangGraph)

## OpenAI LLM 준비 및 퀴즈 파일 지정
* 환경 변수(`.env` 파일)에서 API Key 로딩
* 개발 환경에서는 `gpt-4o-mini` 또는 `gpt-3.5-turbo`
* 핵심 실습 환경이라 `gpt-4o` 사용

In [ ]:
import gradio as gr
import random
import json
import os

from dotenv import load_dotenv
from typing import List, TypedDict

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser
from langgraph.graph import StateGraph, END

from langchain_teddynote.graphs import visualize_graph

# --- 초기 설정 및 데이터 로딩 ---

QUIZ_FILE = "data/quizzes.json"
QUIZ_COUNT = 3
QUIZ_COMMANDS = ["퀴즈", "퀴즈 시작"]

load_dotenv()
llm = ChatOpenAI(model="gpt-4o", temperature=0.5)


## 퀴즈 문항 (`conan_quiz.json`) 로딩
- `def load_quiz()` :	load_quiz라는 이름의 함수를 정의
- `with open(QUIZ_FILE, "r", encoding="utf-8") as f`: 지정된 경로의 JSON 파일(QUIZ_FILE)을 UTF-8 인코딩 방식으로 읽기 모드("r")로 열고, with 구문을 사용하여 파일 사용 후 자동으로 닫히도록 처리
- `all_q = json.load(f)` : JSON 파일 내용을 파싱하여 파이썬 객체(리스트 형태, 퀴즈 문제들)로 변환.
- `return random.sample(all_q, QUIZ_COUNT)` :전체 문제 중 QUIZ_COUNT 개수만큼 무작위로 샘플링하여 반환(중복 없이 선택)

In [ ]:
# 퀴즈 로딩 함수
def load_quiz():
    with open(QUIZ_FILE, "r", encoding="utf-8") as f:
        all_q = json.load(f)
    return random.sample(all_q, QUIZ_COUNT)

## 데이터 구조 정의
### 기본 데이터 정의

`Pydantic`의 `BaseModel`을 상속한 데이터 모델 클래스 정의
- `GradingResult(BaseModel)` : 개별 문제 하나에 대한 채점 결과
    - `Field(description="...")`: description은 LLM에게 이 필드가 무엇을 의미하는지 알려주는 중요한 힌트
- `FinalReport(BaseModel)` : 모든 문제의 채점 결과와 최종 점수를 포함하는 최종 보고서
    - `results: List[GradingResult]`: GradingResult의 리스트(List) 형태
- `llm_with_final_report = llm.with_structured_output(FinalReport)` : LLM의 출력을 FinalReport 클래스의 JSON 형식으로 지정

In [ ]:
class GradingResult(BaseModel):
    """단일 문제에 대한 채점 결과를 상세히 담는 클래스입니다."""
    question: str = Field(description="채점 대상 문제")
    correct_answer: str = Field(description="문제의 정답")
    user_answer: str = Field(description="사용자가 제출한 답변")
    is_correct: bool = Field(description="정답 여부")
    explanation: str = Field(description="정답에 대한 친절한 해설")


class FinalReport(BaseModel):
    """퀴즈의 모든 채점 결과와 최종 점수를 종합한 최종 보고서 클래스입니다."""
    results: List[GradingResult] = Field(description="각 문제별 채점 결과 리스트")
    total_score: str = Field(description="'총점: X/Y' 형식의 최종 점수 요약")

# 출력을 FinalReport 형식으로 지정
llm_with_final_report = llm.with_structured_output(FinalReport)

### 상태 정의
그래프 노드에서 처리되는 상태 클래스로 `QuizState(TypedDict)`는 그래프(workflow)가 관리할 상태 정보
- `chat_history`: Gradio UI 채팅창에 출력되는 사용자 입력과 퀴즈 문항 저장소로 사용자와 LLM이 주고 받는 메시지와 다름.
    - 따라서 `chat_history: Annotated[List[BaseMessage], add_messages]` 형식으로 지정할 수 없음
    - `Annotated`를 사용한다면 `chat_history: Annotated[List[Tuple[str, str]], lambda x, y: x + y]` 형식으로 표현 가능함

In [ ]:

class QuizState(TypedDict):
    """퀴즈 애플리케이션의 전체 상태 - 사용자 입력부터 최종 보고서까지 모든 데이터를 관리"""
    user_input: str
    questions: List[dict]
    user_answers: List[str]
    quiz_index: int
    chat_history: List[tuple]    # UI 채팅창 출력 목록
    grading_input_str: str | None
    final_report: FinalReport | None

## LangGraph 노드 함수 구성
LangGraph 워크플로우에서는 각 단계(노드)가 상태(state)를 받아서, 상태를 갱신한 뒤 반환하는 함수(노드 함수)로 동작
- 입력: state라는 TypedDict 형식의 퀴즈 전체 상태(`QuizState`)를 받음
- 출력: 내부 값들이 갱신된(변경된) 동일 타입의 state 객체를 반환 → LangGraph 노드는 항상 **상태(state) → 상태(state)** 의 형태를 유지


### 퀴즈 시작(상태 초기화) 노드
* 상태 갱신의 목적: "퀴즈 시작"이라는 트리거에서 호출되고 모든 상태 정보를 초기화
    - 문제 리스트 세팅: `state["questions"]`
    - 진행 인덱스 초기화: `state["quiz_index"]`
    - 답변 내역, 리포트 등 모두 초기화
    - 유저(`user`)/AI(`assistant`) 대화 이력(`chat_history`)에 안내 메시지 추가

* `questions = load_quiz()` : 랜덤하게 생성된 QUIZ_COUNT로 개수의 퀴즈 문항
* `state["chat_history"].append()` : UI 채팅창에 출력할 정보 추가

In [ ]:
def start_quiz(state: QuizState) -> QuizState:
    """퀴즈를 시작하고 상태를 초기화합니다."""
    questions = load_quiz()
    if not questions:
        state["chat_history"].append(
            ("assistant", "퀴즈를 불러오는 데 실패했거나 풀 수 있는 문제가 없습니다.")
        )
        state["questions"] = []
        return state

    state["questions"] = questions
    state["quiz_index"] = 0
    state["user_answers"] = []
    state["final_report"] = None
    state["chat_history"].append(("assistant", "명탐정 코난 퀴즈를 시작합니다!"))
    return state

### 문제 출제 노드
* 상태(state) 갱신의 목적 : 문제를 사용자에게 출력(`chat_history`에 포맷팅된 문제 내용을 assistant 메시지로 추가)
    - `quiz_index`에 해당하는 문제(`q`)를 가져와 포맷(문자열 변환)
    - 문제 타입이 `multiple_choice`면, 선택지도 보기 형태로 문자열로 추가
    - `chat_history`에 ("assistant", 문제 텍스트) 튜플을 append

In [ ]:
def ask_question(state: QuizState) -> QuizState:
    """현재 quiz_index에 맞는 문제를 포맷하여 chat_history에 추가합니다."""
    idx = state["quiz_index"]
    q = state["questions"][idx]

    text = f"문제 {idx + 1}: {q['question']}"
    if q["type"] == "multiple_choice":
        choices = [f"{i + 1}. {c}" for i, c in enumerate(q["choices"])]
        text += "\n" + "\n".join(choices)

    state["chat_history"].append(("assistant", text))
    return state

### 사용자 답변 처리 노드
* 상태(state) 갱신의 목적 : 사용자의 답변(`user_input`)을 처리
    - 현재 문제(`quiz_index`)에 맞는 답변을 파싱, 전처리, 정제
    - 선택형 문제(`multiple_choice`)라면 사용자가 번호로 입력했을 때 실제 보기 텍스트로 변환
    - 가공/정제된 답변(`processed_answer`)을 `user_answers` 리스트에 append
    - 진행 인덱스(`quiz_index`)를 +1 하여 다음 문제로 이동할 준비

* `user_input = state["user_input"].strip()` : `state["user_input"]`은 `chat_fn` 에서 입력됨. `strip()`은 앞뒤 공백 모두 제거
* `except (ValueError, IndexError): pass` : 사용자가 "a"처럼 숫자로 변환할 수 없는 값을 입력하거나(ValueError) "5"처럼 선택지 범위를 벗어나는 숫자를 입력하면(IndexError), pass 즉 아무런 작업도 하지 않고 그냥 넘어가서, 사용자가 입력한 값이 `state["user_answers"]`로 저장됨.

In [ ]:
def process_and_store_answer(state: QuizState) -> QuizState:
    """사용자 답변을 처리하고 저장한 뒤, 다음 문제로 넘어갑니다."""
    idx = state["quiz_index"]
    q = state["questions"][idx]
    user_input = state["user_input"].strip()

    # 빈 입력일 경우 안내 메시지만 추가하고, 인덱스는 그대로 유지
    if not user_input:
        state["chat_history"].append(
            ("assistant", "답변을 입력해 주세요."))
        return state

    processed_answer = user_input
    if q["type"] == "multiple_choice":
        try:
            sel = int(user_input) - 1
            if 0 <= sel < len(q["choices"]):
                processed_answer = q["choices"][sel]
        except (ValueError, IndexError):
            pass

    state["user_answers"].append(processed_answer)
    state["quiz_index"] += 1
    return state

### 채점을 위해 LLM에 전달할 프롬프트 생성 노드
* 상태(state) 갱신의 목적 : 
    - 사용자에게 "채점 중" 안내를 위해 `chat_history`에 추가
    - 채점 대상 데이터를 `state["grading_input_str"]`에 저장 → 다음 노드(LLM 채점 호출 등)에서 활용
* `zip(state["questions"], state["user_answers"])` : 각 문제와 각 답변을 쌍으로 묶어서 반환 (튜플: (문제, 답변))
* `enumerate(...)` : 튜플 쌍에 인덱스 번호(`i`)를 붙여서 (`i, (q, a)`) 형태로 반환

In [ ]:
def prepare_grading_prompt(state: QuizState) -> QuizState:
    """채점을 위해 LLM에 전달할 프롬프트를 생성합니다."""
    state["chat_history"].append(
        ("assistant", "채점을 진행합니다..."))
        
    parts = [
        "지금부터 아래의 문제와 정답, 그리고 사용자의 답변을 보고 채점을 시작해주세요."
    ]
    for i, (q, a) in enumerate(zip(state["questions"], state["user_answers"])):
        parts.append(f"\n--- 문제 {i + 1} ---")
        parts.append(f"문제: {q['question']}")
        if q["type"] == "multiple_choice":
            parts.append(f"선택지: {', '.join(q['choices'])}")
        parts.append(f"정답: {q['answer']}")
        parts.append(f"사용자 답변: {a}")

    state["grading_input_str"] = "\n".join(parts)
    return state

### LLM 채점 및 파싱 노드
* 상태(state) 갱신의 목적 : 
    - 채점 대상 데이터를 `state["grading_input_str"]`를 LLM에 전달
    - LLM의 채점 결과를 FinalReport 클래스 JSON 데이터로 수신하여 `state["final_report"]`에 저장
    
- `chain = prompt | llm_with_final_report` : 프롬프트와 FinalReport를 출력으로 하는 LLM을 LCEL 체인으로 연결



In [ ]:
def grade_with_llm_and_parse(state: QuizState) -> QuizState:
    """LLM을 호출하여 채점하고 결과를 파싱합니다."""
    system_message = """
    당신은 '명탐정 코난' 퀴즈의 전문 채점관입니다. 주어진 문제, 정답, 사용자 답변을 바탕으로 채점해주세요. 
    각 문제에 대해 정답 여부를 판단하고 친절한 해설을 덧붙여주세요. 
    모든 채점이 끝나면, 마지막에는 '총점: X/Y' 형식으로 최종 점수를 반드시 요약해서 보여줘야 합니다. 
    반드시 지정된 JSON 형식으로만 답변해야 합니다."""

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_message),
            ("human", "{grading_data}"),
        ]
    )

    try:
        # 체인 생성
        chain = prompt | llm_with_final_report
        report = chain.invoke({"grading_data": state["grading_input_str"]})
        state["final_report"] = report
    except Exception as e:
        print(f"채점 중 오류 발생: {e}")
        error_report = FinalReport(results=[], total_score="채점 오류가 발생했습니다.")
        state["final_report"] = error_report

    return state

### 최종 리포트(FinalReport)를 사용자에게 보여줄 문자열로 변환 노드
상태(state) 갱신의 목적 : LLM이 제공한 최종 채점 결과를 문제별로 정오, 정답, 제출 답변, 해설을 보기 좋게 포매팅
- 마지막에 총점도 포함
- 안내 메시지(재시작 방법 등)도 추가
- 결과를 `assistant` 메시지로 `chat_history`에 append

워크플로우 마지막 단계로 구조화된 채점 결과 객체(`final_report_obj`)를 사람이 읽을 수 있는 문자열로 변환하여 `chat_history`에 추가함으로써 사용자에게 "마지막 답변"을 제공

In [ ]:
def format_final_report(state: QuizState) -> QuizState:
    """파싱된 최종 리포트를 사용자에게 보여줄 문자열로 변환합니다."""
    final_report_obj = state["final_report"]
    report_parts = ["채점이 완료되었습니다! 🎉\n"]

    if final_report_obj and final_report_obj.results:
        for i, res in enumerate(final_report_obj.results):
            is_correct_text = "✅ 정답" if res.is_correct else "❌ 오답"
            report_parts.append(f"--- 문제 {i + 1} ---")
            report_parts.append(f"문제: {res.question}")
            report_parts.append(f"정답: {res.correct_answer}")
            report_parts.append(f"제출한 답변: {res.user_answer}")
            report_parts.append(f"결과: {is_correct_text}")
            report_parts.append(f"해설: {res.explanation}\n")
        report_parts.append(f"**{final_report_obj.total_score}**")
    else:
        report_parts.append("채점 결과를 생성하는 데 실패했습니다.")

    report_parts.append("\n퀴즈를 다시 시작하려면 '퀴즈 시작'이라고 입력해주세요.")
    state["chat_history"].append(("assistant", "\n".join(report_parts)))
    return state

### 퀴즈 시작 명령어 예외 처리 노드
사용자가 “퀴즈 시작” 명령어를 입력하지 않은 경우, 올바른 명령어 사용법(트리거 문구)을 안내하는 메시지를 `chat_history`에 `assistant` 역할로 추가

In [ ]:
def handle_invalid_start(state: QuizState) -> QuizState:
    """퀴즈 시작 명령어가 아닐 경우 안내 메시지를 추가합니다."""
    help_message = "'퀴즈' 또는 '퀴즈 시작'이라고 입력하면 퀴즈가 시작됩니다."
    state["chat_history"].append(("assistant", help_message))
    return state

### StateGraph 조건부 함수
분기(`Edge Condition`) 함수로, 입력(state)을 분석해서 다음 실행할 노드 이름을 반환
- `should_continue_quiz` : 퀴즈가 아직 남아 있는지, 아니면 채점 단계로 넘어가야 하는지 다음 노드 이름을 결
- `route_initial_input` : 사용자의 첫 입력이 이미 퀴즈를 진행 중인 상태인지, “퀴즈 시작” 명령어인지, 또는 잘못된 입력인지 구분하여 시작 노드를 결정

In [ ]:
def should_continue_quiz(state: QuizState) -> str:
    """퀴즈를 계속할지, 채점을 시작할지 결정합니다."""
    if state["quiz_index"] < len(state["questions"]):
        return "continue_quiz"
    else:
        return "grade_quiz"


def route_initial_input(state: QuizState) -> str:
    """사용자의 입력을 분석하여 워크플로우의 시작점을 결정합니다."""
    if state.get("questions") and state["questions"]:
        return "process_answer"
    else:
        if state["user_input"].strip().lower() in QUIZ_COMMANDS:
            return "start_quiz"
        else:
            return "invalid_start"

## StateGraph 정의 및 컴파일

### 노드 추가
- 각 노드는 하나의 “기능적 단계”를 담당 : `start_quiz`는 상태 초기화, `ask_question`은 문제 출제, `process_answer`는 답변 처리 등
- `invalid_start`는 잘못된 시작 입력 안내

### 조건부 진입점 설정
- 워크플로우의 초기 입력을 분석하여 “퀴즈 시작” 명령이면 `start_quiz`, 이미 진행 중이면 `process_answer`, 그 외엔 `invalid_start`로 각기 다른 노드에서 실행이 시작됨

### 노드 연결(엣지 설정)

직접 연결:
- `start_quiz` → `ask_question`: 퀴즈 시작 후 문제 출제
- `prepare_grading` → `grade_and_parse` → `format_report`: 채점 준비→실행→리포트 출력
- `format_report` → `END`: 채점 결과 출력 후 종료

조건부 연결:
- `process_answer` 다음에 `should_continue_quiz`로 검사, 문제 남아 있으면 `ask_question`, 다 풀었으면 `prepare_grading`
- 노드 종료 연결 : `ask_question` → `END`, `invalid_start` → `END` (`ask_question → END`는 “한 문제만 내는” 임시 흐름처럼 보이니, 여러 문제 반복엔 반드시 위 조건부 연결이 필요!)

In [ ]:
workflow = StateGraph(QuizState)

# 노드 추가
workflow.add_node("start_quiz", start_quiz)
workflow.add_node("ask_question", ask_question)
workflow.add_node("process_answer", process_and_store_answer)
workflow.add_node("prepare_grading", prepare_grading_prompt)
workflow.add_node("grade_and_parse", grade_with_llm_and_parse)
workflow.add_node("format_report", format_final_report)
workflow.add_node("invalid_start", handle_invalid_start)

# 조건부 진입점 설정
workflow.set_conditional_entry_point(
    route_initial_input,
    {
        "start_quiz": "start_quiz",
        "process_answer": "process_answer",
        "invalid_start": "invalid_start",
    },
)

# 엣지 연결
workflow.add_edge("start_quiz", "ask_question")
workflow.add_edge("ask_question", END)
workflow.add_edge("invalid_start", END)

workflow.add_conditional_edges(
    "process_answer",
    should_continue_quiz,
    {"continue_quiz": "ask_question", "grade_quiz": "prepare_grading"},
)
workflow.add_edge("prepare_grading", "grade_and_parse")
workflow.add_edge("grade_and_parse", "format_report")
workflow.add_edge("format_report", END)

quiz_app = workflow.compile()

## 그래프 시각화

In [ ]:
visualize_graph(quiz_app)

## UI 인터페이스 함수
`init_state()` :
- 앱을 초기화할 때 사용할 빈 상태(딕셔너리) 반환
- `quiz_state` 아래에 문제/대화 이력만 먼저 준비
- 다른 필드는 워크플로우 도중 필요에 따라 추가됨(예: quiz_index, user_answers, final_report 등)

`chat_fn(user_input, state)` : Gradio 챗봇 UI에서 실제로 호출되는 메인 처리 함수


In [ ]:
def init_state():
    return {"quiz_state": {"questions": [], "chat_history": []}}


def chat_fn(user_input, state):
    quiz_state = state["quiz_state"]

    if quiz_state.get("final_report") and user_input.strip().lower() in QUIZ_COMMANDS:
        quiz_state = init_state()["quiz_state"]

    current_chat_history = quiz_state.get("chat_history", [])
    current_chat_history.append(("user", user_input))

    graph_input = {
        **quiz_state,
        "user_input": user_input,
        "chat_history": current_chat_history,
    }

    new_state = quiz_app.invoke(graph_input)

    state["quiz_state"] = new_state

    chat_display = [
        {"role": role, "content": content}
        for role, content in new_state["chat_history"]
    ]

    return chat_display, state

## Gradio UI 정의
Gradio의 Blocks UI 구성 방식을 사용해 명탐정 코난 퀴즈 챗봇 인터페이스를 구성

In [ ]:
# Gradio UI
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("### 🕵️ 명탐정 코난 매니아 판별기 (by LangGraph)")

    chatbot = gr.Chatbot(
        label="명탐정 코난 퀴즈 챗봇",
        height=400,
        avatar_images=("data/avatar_user.png", "data/avatar_conan.png"),
        type="messages",
    )

    txt = gr.Textbox(placeholder="'퀴즈 시작'을 입력해보세요!", show_label=False)
    state = gr.State(init_state())

    txt.submit(chat_fn, inputs=[txt, state], outputs=[chatbot, state])
    txt.submit(lambda: "", None, txt)

    demo.launch()

-----
** End of Documents **